In [1]:
import requests
import pandas as pd
import datetime
import json
import re
from amz_local_search.resources.fba_centers import fba_centers_info

/Users/evgenyp/Projects/AmzJet-ZT/zt-amzjet-backend/amz-deploy/dev/.venv-dev/lib/python2.7/site-packages/OpenSSL/crypto.py:12: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in a future release.
  from cryptography import x509


In [2]:
ZONTRACKER_API_USERNAME = 'amzjet'
ZONTRACKER_API_PASSWORD = '84K41VH1Uj'

In [7]:
import bb_utils.time.utils as tu

email = 'veronica.aguirrebeitia@gmail.com'

utc_time = tu.utc_time()
start_date_s = (utc_time - datetime.timedelta(days=3)).strftime('%Y-%m-%d')
end_date_s = (utc_time + datetime.timedelta(days=3)).strftime('%Y-%m-%d')

inventory_api_url_template = 'https://app.zontracker.io/api/inventory/%s/%s/%s'
inventory_api_url = inventory_api_url_template % (email, start_date_s,
                                                  end_date_s)

In [8]:
session = requests.Session()
session.auth = (ZONTRACKER_API_USERNAME, ZONTRACKER_API_PASSWORD)

api_response = session.get(inventory_api_url)
api_response.raise_for_status()
raw_inventory_data = json.loads(api_response.content)

In [10]:
inventory_df = pd.DataFrame(raw_inventory_data.get('data', []))
inventory_df.head(3)

,detailed_disposition,fnsku,fulfillment_center_id,name,quantity,seller_id,sku,timestamp
0,SELLABLE,X002DKEUT3,TUL2,"Silicone Placemat for Kids, Babies and Toddler...",4,A1CXKW6PG6HFUE,HEDWHITE,1604102400
1,SELLABLE,X002DKEUT3,TPA1,"Silicone Placemat for Kids, Babies and Toddler...",3,A1CXKW6PG6HFUE,HEDWHITE,1604102400
2,SELLABLE,X002DKEUT3,SMF1,"Silicone Placemat for Kids, Babies and Toddler...",1,A1CXKW6PG6HFUE,HEDWHITE,1604102400


In [11]:
inventory_df.describe(include=['object'])

,detailed_disposition,fnsku,fulfillment_center_id,name,seller_id,sku
count,148,148,148,148,148,148
unique,2,11,28,9,1,11
top,SELLABLE,X002B649RH,FTW6,Silicone Placemat for Kids and Babies | Ideal ...,A1CXKW6PG6HFUE,REINRED
freq,146,28,16,28,148,28


In [12]:
inventory_df['state'] = inventory_df['fulfillment_center_id'].map(fba_centers_info.code_to_state)

In [13]:
by_fnsku = inventory_df.groupby('fnsku')
latest_inventory_df = inventory_df[inventory_df['timestamp'] == by_fnsku['timestamp'].
                                 transform('max')]
latest_inventory_df.head(3)

,detailed_disposition,fnsku,fulfillment_center_id,name,quantity,seller_id,sku,timestamp,state
74,SELLABLE,X002DKEUT3,TUL2,"Silicone Placemat for Kids, Babies and Toddler...",4,A1CXKW6PG6HFUE,HEDWHITE,1604188800,KS
75,SELLABLE,X002DKEUT3,TPA1,"Silicone Placemat for Kids, Babies and Toddler...",3,A1CXKW6PG6HFUE,HEDWHITE,1604188800,FL
76,SELLABLE,X002DKEUT3,SMF1,"Silicone Placemat for Kids, Babies and Toddler...",1,A1CXKW6PG6HFUE,HEDWHITE,1604188800,CA


In [14]:
latest_inventory_df.groupby('state')['quantity'].sum().head(100)

state
AL     5
CA    18
CT    17
DE     1
FL    28
IL     7
KS    25
KY     1
NC     1
NJ     6
NV     7
NY     2
OH     9
OK     5
TX    48
UT     8
VA     1
Name: quantity, dtype: int64

In [15]:
from amz_local_search.resources.fba_centers import fba_centers_info

In [16]:
latest_inventory_df.to_dict(orient='records')

[{u'detailed_disposition': u'SELLABLE',
  u'fnsku': u'X002DKEUT3',
  u'fulfillment_center_id': u'TUL2',
  u'name': u'Silicone Placemat for Kids, Babies and Toddlers | Non-Slip Mats, Chemical Free & Food Grade Baby Mat | Portable Placemats | Raised Edges Prevent Spills | BamBamBoom Hedgehog-Dino (White)',
  u'quantity': 4,
  u'seller_id': u'A1CXKW6PG6HFUE',
  u'sku': u'HEDWHITE',
  'state': 'KS',
  u'timestamp': 1604188800},
 {u'detailed_disposition': u'SELLABLE',
  u'fnsku': u'X002DKEUT3',
  u'fulfillment_center_id': u'TPA1',
  u'name': u'Silicone Placemat for Kids, Babies and Toddlers | Non-Slip Mats, Chemical Free & Food Grade Baby Mat | Portable Placemats | Raised Edges Prevent Spills | BamBamBoom Hedgehog-Dino (White)',
  u'quantity': 3,
  u'seller_id': u'A1CXKW6PG6HFUE',
  u'sku': u'HEDWHITE',
  'state': 'FL',
  u'timestamp': 1604188800},
 {u'detailed_disposition': u'SELLABLE',
  u'fnsku': u'X002DKEUT3',
  u'fulfillment_center_id': u'SMF1',
  u'name': u'Silicone Placemat for Kids,

In [17]:
latest_inventory_df[latest_inventory_df.fulfillment_center_id=='OKC1']

,detailed_disposition,fnsku,fulfillment_center_id,name,quantity,seller_id,sku,timestamp,state
90,SELLABLE,X002CFXH3T,OKC1,"Silicone Placemat for Kids, Babies and Toddler...",1,A1CXKW6PG6HFUE,UNISLATE,1604188800,OK
104,SELLABLE,X002B649RH,OKC1,Silicone Placemat for Kids and Babies | Ideal ...,1,A1CXKW6PG6HFUE,REINRED,1604188800,OK
117,SELLABLE,X002B5Z4EZ,OKC1,"Silicone Placemat for Kids, Babies and Toddler...",1,A1CXKW6PG6HFUE,HEDBLCK,1604188800,OK
125,SELLABLE,X002B3SOP3,OKC1,"Silicone Placemat for Kids, Babies and Toddler...",1,A1CXKW6PG6HFUE,WV-8IEX-YUOK,1604188800,OK
131,SELLABLE,X0029LKR0R,OKC1,"Silicone Placemat for Kids, Babies and Toddler...",1,A1CXKW6PG6HFUE,UC0005,1604188800,OK


In [18]:
latest_inventory_df[latest_inventory_df.state.map(lambda s: s is None or len(s) != 2)]['fulfillment_center_id'].unique()

array([u'GEG1', u'PSP1', u'BFL1', u'*XFR'], dtype=object)

In [19]:
latest_inventory_df.groupby('state')

In [20]:
latest_inventory_df.groupby('state')['quantity'].sum().reset_index().to_dict(
            orient='records')

[{'quantity': 5, 'state': 'AL'},
 {'quantity': 18, 'state': 'CA'},
 {'quantity': 17, 'state': 'CT'},
 {'quantity': 1, 'state': 'DE'},
 {'quantity': 28, 'state': 'FL'},
 {'quantity': 7, 'state': 'IL'},
 {'quantity': 25, 'state': 'KS'},
 {'quantity': 1, 'state': 'KY'},
 {'quantity': 1, 'state': 'NC'},
 {'quantity': 6, 'state': 'NJ'},
 {'quantity': 7, 'state': 'NV'},
 {'quantity': 2, 'state': 'NY'},
 {'quantity': 9, 'state': 'OH'},
 {'quantity': 5, 'state': 'OK'},
 {'quantity': 48, 'state': 'TX'},
 {'quantity': 8, 'state': 'UT'},
 {'quantity': 1, 'state': 'VA'}]

In [22]:
import bb_utils.time.utils as tu
latest_inventory_df['datetime'] = latest_inventory_df['timestamp'].map(tu.from_unix_seconds)
latest_inventory_df.to_csv('/Users/evgenyp/Downloads/anuj2911-all.csv')

/Users/evgenyp/Projects/AmzJet/AmzJet/amz-deploy/dev/.venv-dev/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
